<H1>MAPK8IP3 genotype phenotype correlations</H1>

In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath('../src'))

from genophenocorr import *
import glob
import pprint
print(f"Using genophenocorr version {genophenocorr.__version__}")

NameError: name 'genophenocorr' is not defined

In [3]:
allPatients = Cohort('phenopackets/*.json')

WARNING - Current Protein ID or Transcript ID does not match with Described Isoform
WARNING - Current Protein ID or Transcript ID does not match with Described Isoform


In [4]:
allPatients.get_cohort_description_df()

,Patient ID,Disease,Gene,Variant,Protein,HPO Terms
0,12,618443,{MRPS34},{chr16:g.1767834C>T},{NP_076425.1},"{HP:0010864, HP:0100021, HP:0001263, HP:0001257}"
1,2,None,{LOC107984011},{chr9:g.70598463C>T},{None},"{HP:0001252, HP:0000750, HP:0012469, HP:003193..."
2,3,None,{LOC107984011},{chr9:g.70598463C>T},{None},"{HP:0001252, HP:0002069, HP:0000750, HP:000072..."
3,13,618443,{MRPS34},{chr16:g.1767834C>T},{NP_076425.1},"{HP:0001252, HP:0100704, HP:0007301, HP:000237..."
4,1,618443,{JPT2},{chr16:g.1706404del},{NP_653171.1},"{HP:0001251, HP:0001263, HP:0001252, HP:0000717}"
5,4,None,{LOC107984011},{chr9:g.70598463C>T},{None},"{HP:0002133, HP:0001252, HP:0010864, HP:000072..."
6,PMID_111_probandA,OMIM:618443,{MAPK8IP3},{chr16:g.1762846C>T},{NP_001305781.1},"{HP:0033725, HP:0002104, HP:0001662, HP:000126..."
7,8,None,{KLF9-DT},{chr9:g.70553229G>T},{None},"{HP:0001252, HP:0011147, HP:0000750, HP:003193..."
8,6,618443,{MAPK8IP3},{chr16:g.1760409T>C},{NP_001305781.1},"{HP:0001263, HP:0001256, HP:0001252}"
9,7,618443,{MAPK8IP3},{chr16:g.1762388G>A},{NP_001305781.1},"{HP:0001263, HP:0001256}"


In [12]:
for var in allPatients.all_variants_d.values():
    pprint.pprint(var._variant_json)

{'allele_string': 'C/T',
 'assembly_name': 'GRCh38',
 'colocated_variants': [{'allele_string': 'C/T',
                         'clin_sig': ['likely_pathogenic', 'pathogenic'],
                         'clin_sig_allele': 'T:pathogenic;T:pathogenic/likely_pathogenic',
                         'end': 1767834,
                         'id': 'rs1567214097',
                         'phenotype_or_disease': 1,
                         'pubmed': [30945334, 30612693],
                         'seq_region_name': '16',
                         'start': 1767834,
                         'strand': 1,
                         'var_synonyms': {'ClinVar': ['RCV000779605',
                                                      'VCV000632565',
                                                      'RCV001266867'],
                                          'OMIM': [605431.0005],
                                          'UniProt': ['VAR_082615']}}],
 'end': 1767834,
 'id': 'chr16:g.1767834C>T',
 'input': '